<a href="https://colab.research.google.com/github/anikpatra/Ridesharing/blob/master/TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plot
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

In [4]:
tf.__version__

'1.15.0'

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data")

In [0]:
data = mnist

In [12]:
print("Size of:")
print("- Training-set:\t\t{}".format(data.num_train))
print("- Validation-set:\t{}".format(data.num_val))
print("- Test-set:\t\t{}".format(data.num_test))

Size of:


AttributeError: ignored


Classification

In [1]:
import pandas as pd
census = pd.read_csv("census_data.csv")

FileNotFoundError: ignored

In [0]:
url='https://raw.githubusercontent.com/HarshSingh16/Tensorflow/master/census_data_Classification.csv'
df = pd.read_csv(url)

In [4]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
census=df
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def labelfix(label):
  if(label=='<=50K'):
    return 0
  else:
    return 1

In [0]:
census['income_bracket']=census['income_bracket'].apply(labelfix)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_data=census.drop('income_bracket',axis=1)
y_label=census['income_bracket']
x_train,x_test,y_train,y_test=train_test_split(x_data,y_label,test_size=0.3,random_state=101)

In [11]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [0]:
import tensorflow as tf

In [0]:
gender=tf.feature_column.categorical_column_with_vocabulary_list("gender",["female","male"])
occupation=tf.feature_column.categorical_column_with_hash_bucket("occupation",hash_bucket_size=1000)
education=tf.feature_column.categorical_column_with_hash_bucket("education",hash_bucket_size=1000)
marital_status=tf.feature_column.categorical_column_with_hash_bucket("marital_status",hash_bucket_size=1000)
workclass=tf.feature_column.categorical_column_with_hash_bucket("workclass",hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket("relationship",hash_bucket_size=1000)
native_country=tf.feature_column.categorical_column_with_hash_bucket("native_country",hash_bucket_size=1000)

In [0]:
age=tf.feature_column.numeric_column("age")
education_num=tf.feature_column.numeric_column("education_num")
capital_gain=tf.feature_column.numeric_column("capital_gain")
capital_loss=tf.feature_column.numeric_column("capital_loss")
hours_per_week=tf.feature_column.numeric_column("hours_per_week")

In [0]:
feat_cols=[gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [0]:
input_func=tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [26]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjktdmuqa', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa9dc74a5f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
model.train(input_fn=input_func,steps=5000)

In [0]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)

In [30]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjktdmuqa/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [32]:
predictions[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'logistic': array([1.], dtype=float32),
 'logits': array([21.67251], dtype=float32),
 'probabilities': array([3.870334e-10, 1.000000e+00], dtype=float32)}

In [33]:
predictions[0]['class_ids'][0]

1

In [34]:
final_preds[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      9769

    accuracy                           1.00      9769
   macro avg       1.00      1.00      1.00      9769
weighted avg       1.00      1.00      1.00      9769



In [0]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,batch_size=len(x_test),shuffle=False)

In [37]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      9769

    accuracy                           1.00      9769
   macro avg       1.00      1.00      1.00      9769
weighted avg       1.00      1.00      1.00      9769



In [38]:
results = model.evaluate(eval_input_func)
results

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-02T12:13:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjktdmuqa/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-02-12:13:18
INFO:tensorflow:Saving dict for global step 5000: accuracy = 1.0, accuracy_baseline = 1.0, auc = 0.0, auc_precision_recall = 1.0, average_loss = 2.4378937e-06, global_step = 5000, label/mean = 1.0, loss = 0.023815783, precision = 1.0, prediction/mean = 0.9999978, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpjktdmuqa/model.ckpt-5000


{'accuracy': 1.0,
 'accuracy_baseline': 1.0,
 'auc': 0.0,
 'auc_precision_recall': 1.0,
 'average_loss': 2.4378937e-06,
 'global_step': 5000,
 'label/mean': 1.0,
 'loss': 0.023815783,
 'precision': 1.0,
 'prediction/mean': 0.9999978,
 'recall': 1.0}